### 1. combine `mon_features.pkl` & `unmon_features.pkl` into `features_df`

In [1]:
import pandas as pd
import numpy as np
import pickle

MON_FILE_PATH = '/content/mon_features.pkl'
UNMON_FILE_PATH = '/content/unmon_features.pkl'

LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER1 = [
    'total_transmission_time', 'cumul_packets_10pct',
    'cumul_packets_30pct', 'cumul_max',
    'num_incoming_first_30', 'outgoing_first_30',
    'avg_incoming_order_first_30', 'avg_outgoing_order_first_30'
]

mon_features_df = pd.read_pickle(MON_FILE_PATH)
unmon_features_df = pd.read_pickle(UNMON_FILE_PATH)

features_df = pd.concat([mon_features_df, unmon_features_df], ignore_index=True)

X = features_df[FEATURES_VER1]
y = features_df[LABEL_COLUMN[1]]

print(X)
print(y)

       total_transmission_time  cumul_packets_10pct  cumul_packets_30pct  \
0                        10.14                 23.0                 55.0   
1                        10.16                  5.0                 49.0   
2                        11.11                  8.0                 34.0   
3                        13.36                 15.0                 57.0   
4                        10.64                 22.0                 53.0   
...                        ...                  ...                  ...   
28995                    32.09                 24.0               1941.0   
28996                    38.62                  9.0               1656.0   
28997                    34.93                240.0                357.0   
28998                    11.84                  7.0                 28.0   
28999                     9.62                  4.0                  8.0   

       cumul_max  num_incoming_first_30  outgoing_first_30  \
0            0.0         

### 2. SVM

Ignore ConvergenceWarning

In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

Split the dataset into training and testing sets

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Map all points to have mean=0 and std=1

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train.values)
X_test_scale = scaler.transform(X_test.values)
X_scale = scaler.fit_transform(X.values)

Train and test SVM using RBF kernel

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

clf_rbf = SVC(kernel='rbf', C=100, gamma=0.1, class_weight='balanced')
clf_rbf.fit(X_train_scale, y_train)
y_pred_rbf = clf_rbf.predict(X_test_scale)

print("============ Before Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_rbf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rbf))

============ Before Hyperparameter tuning ============
SVM Accuracy: 0.9332413793103448
Classification Report:
               precision    recall  f1-score   support

          -1       0.87      0.95      0.91      2470
           1       0.97      0.93      0.95      4780

    accuracy                           0.93      7250
   macro avg       0.92      0.94      0.93      7250
weighted avg       0.94      0.93      0.93      7250



### 3. Hyperparameter tuning by using Grid Search

In [6]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

search = HalvingGridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train_scale, y_train)
y_pred_tuned = search.predict(X_test_scale)

print("============ After Hyperparameter tuning ============")
print("SVM Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))

============ After Hyperparameter tuning ============
SVM Accuracy: 0.9388965517241379
Classification Report:
               precision    recall  f1-score   support

          -1       0.88      0.95      0.91      2470
           1       0.97      0.94      0.95      4780

    accuracy                           0.94      7250
   macro avg       0.93      0.94      0.93      7250
weighted avg       0.94      0.94      0.94      7250

